# Substrate fringe handling in MAGIC

**Copyright 2025 Martin Köhler**

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

## Example: single_plate_100um_x_100um_li1_over_substrate

![title]("../figures/test_patterns/single_plate_100um_x_100um_li1_over_substrate_-_perimeter.png")

MAGIC extraction of the example https://github.com/martinjankoehler/klayout-pex/blob/main/testdata/designs/sky130A/test_patterns/single_plate_100um_x_100um_li1_over_substrate.gds.gz

```
Magic 8.3 revision 486

CapDebug (ExtTechSimplePerimCap) (li-space): exts_perimCap[90][0]=40.700000
...

CapDebug (extNodeAreaFunc/Area) layer li(90), net li_0_0#, area=400000000 (10000 µm^2) nreg_cap += 369.9 fF
CapDebug (extNodeAreaFunc/Perimeter/TopSide) layer li(90), net li_0_0#, length=20000 (100 µm), nreg_cap += 4.07 fF (now nreg_cap = 373.97 fF)
CapDebug (extNodeAreaFunc/Perimeter/LeftSide) layer li(90), net li_0_0#, length=20000 (100 µm), nreg_cap += 4.07 fF (now nreg_cap = 378.04 fF)
CapDebug (extNodeAreaFunc/Perimeter/BottomSide) layer li(90), net li_0_0#, length=20000 (100 µm), nreg_cap += 4.07 fF (now nreg_cap = 382.11 fF)
CapDebug (extNodeAreaFunc/Perimeter/RightSide) layer li(90), net li_0_0#, length=20000 (100 µm), nreg_cap += 4.07 fF (now nreg_cap = 386.18 fF)
CapDebug (extSetResist): li_0_0# area=400000000 (10000 µm^2) perim=80000 (400 µm)
CapDebug ---
```

Magic has two fringe calculations:
- perimeter (assumes $\infty$ halo)
- sideoverlap (assumes configured halo, e.g. $8.0\ µm$) and also uses overlap capacitance coefficients

Using the fringe calculation instead of perimeter calculation, then MAGIC has the issue that only if the halo is large enough, cfrac can become $1.0$

In [22]:
import math

overlap_li_substrate = 36.99
perimcap_li_substrate = 40.7  # comes from tech file

def print_cap(halo_um: float):
    scaling = 0.02 * 0.01 * 0.5 * 200.0   # MAGIC Scaling = 1e-4
    side_length_um = 100.0  # µm

    distance_um = halo_um

    alpha_c = overlap_li_substrate * scaling
    cfrac = (2.0 / math.pi) * math.atan(alpha_c * distance_um)
    one_side_cap_fF = cfrac * side_length_um * perimcap_li_substrate / 1000.0

    print(f"one_side_cap is {one_side_cap_fF} fF\n"
          f"   - halo = {halo_um} µm\n"
          f"   - alpha_c = {alpha_c}\n"
          f"   - cfrac = {cfrac}\n")

print_cap(halo_um=8.0)     # this is off
print_cap(halo_um=10000.0) # this is correct


one_side_cap is 3.636301844883797 fF
   - halo = 8.0 µm
   - alpha_c = 0.7398
   - cfrac = 0.8934402567282057

one_side_cap is 4.069649764469079 fF
   - halo = 10000.0 µm
   - alpha_c = 0.7398
   - cfrac = 0.9999139470439997



## 

